In [38]:
import altair as alt
import polars as pl
from pathlib import Path
from vega_datasets import data
import geopandas as gpd
import pandas as pd
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [39]:
# Load data set
df = pl.read_csv("../data/FoodImports.csv", ignore_errors=True)

# Clean data set
df_clean = (
    df
    .filter(
        ~pl.col("Country").str.contains("WORLD|Rest of World|Quantity")
        & (pl.col("UOM") == "Million $")
        & pl.col("FoodValue").is_not_null()
        & pl.col("YearNum").is_not_null()
    )
    .filter(~pl.col("Commodity").str.starts_with("Total"))
)
df_clean_b = df_clean.with_columns(
    (pl.col("FoodValue") / 1000).alias("FoodValue_B")
)

In [40]:
inflation_data = [
    {"YearNum": 1999, "CPI": 166.6},
    {"YearNum": 2000, "CPI": 172.2},
    {"YearNum": 2001, "CPI": 177.1},
    {"YearNum": 2002, "CPI": 179.8},
    {"YearNum": 2003, "CPI": 184},
    {"YearNum": 2004, "CPI": 188.9},
    {"YearNum": 2005, "CPI": 195.3},
    {"YearNum": 2006, "CPI": 201.6},
    {"YearNum": 2007, "CPI": 207.3},
    {"YearNum": 2008, "CPI": 214.5},
    {"YearNum": 2009, "CPI": 214.7},
    {"YearNum": 2010, "CPI": 218.1},
    {"YearNum": 2011, "CPI": 224.9},
    {"YearNum": 2012, "CPI": 229.6},
    {"YearNum": 2013, "CPI": 232.9},
    {"YearNum": 2014, "CPI": 236.8},
    {"YearNum": 2015, "CPI": 237.0},
    {"YearNum": 2016, "CPI": 240.0},
    {"YearNum": 2017, "CPI": 245.1},
    {"YearNum": 2018, "CPI": 251.0},
    {"YearNum": 2019, "CPI": 255.3},
    {"YearNum": 2020, "CPI": 258.8},
    {"YearNum": 2021, "CPI": 270.0},
    {"YearNum": 2022, "CPI": 287.5},
    {"YearNum": 2023, "CPI": 304.7},
    {"YearNum": 2024, "CPI": 294.5},
]

df_inflation = pl.DataFrame(inflation_data)

In [41]:
df_adjusted = df_clean_b.join(df_inflation, on="YearNum", how="left")

base_cpi = df_inflation.filter(pl.col("YearNum") == 2024)["CPI"][0]

df_adjusted = df_adjusted.with_columns(
    (
        pl.col("FoodValue_B") * (base_cpi / pl.col("CPI"))
    ).alias("FoodValue_B_Real")
)

chart_import_trend = (
    alt.Chart(df_adjusted)
    .mark_line(point=True)
    .encode(
        x=alt.X("YearNum:O", title="Year"), 
        y=alt.Y(
            "sum(FoodValue_B):Q", 
            title="Total Import Value (Billion $)"
        )
    )
    .properties(title={
                "text": "Total US Food Imports Over Time (1999 - 2024)",
                "subtitle": "Values adjusted for inflation using U.S. CPI (base year = 2024)"
                }
    )
)

chart_import_trend

alt.Chart(...)

In [42]:
df_country_totals = (
    df_clean_b
    .group_by("Country")
    .agg(pl.sum("FoodValue_B").alias("TotalImports_B"))
    .sort("TotalImports_B", descending=True)
)

df_top8 = df_country_totals.head(8)

custom_colors = {
    "MEXICO": "#006847",
    "CANADA": "#C8102E",
    "CHINA": "#EE1C25",
    "INDIA": "#FF9933",
    "FRANCE": "#0055A4",
    "ITALY": "#009246",
    "CHILE": "#0033A0",
    "INDONESIA": "#D00000",
}

chart_top_exporting_countries = (
    alt.Chart(df_top8)
    .mark_bar()
    .encode(
        x=alt.X("TotalImports_B:Q", title="Accumulated Total Food Imports from 1999 to 2024 (Billion $)"),
        y=alt.Y("Country:N", sort='-x', title="Country"),
        color=alt.Color(
            "Country:N",
            scale=alt.Scale(
                domain=list(custom_colors.keys()),
                range=list(custom_colors.values())
            ),
            legend=None
        )
    )
    .properties(
        title=f"Top Exporting Countries to the US"
    )
)

chart_top_exporting_countries

alt.Chart(...)

In [43]:
df_can_mex = (
    df_clean_b
    .filter(pl.col("Country").is_in(["CANADA", "MEXICO"]))
    .group_by(["YearNum", "Country"])
    .agg(pl.sum("FoodValue_B").alias("TotalImports_B"))
    .sort(["Country", "YearNum"])
)

df_can_mex_adj = (
    df_can_mex.join(df_inflation, on="YearNum", how="left")
    .with_columns(
        (pl.col("TotalImports_B") * (294.5 / pl.col("CPI"))).alias("TotalImports_Real_B")
    )
)


In [44]:
chart_can_mex = (
    alt.Chart(df_can_mex_adj)
    .mark_line(point=True)
    .encode(
        x=alt.X("YearNum:O", title="Year"),
        y=alt.Y("TotalImports_B:Q", title="Total Import Value (Billion $)"),
        color=alt.Color("Country:N", title="Country", scale=alt.Scale(domain=["CANADA", "MEXICO"], range=["#C8102E", "#006847"])),
    )
    .properties(
        title={
            "text": "Canada vs Mexico: U.S. Food Import Trends Over Time (1999 - 2024)",
            "subtitle": "Values adjusted for inflation using U.S. CPI (base year = 2024)"
        }
    )
)

rule = (
    alt.Chart(alt.Data(values=[{"YearNum": 2016}]))
    .mark_rule(color="gray")
    .encode(x="YearNum:O")
)

chart_can_mex + rule

alt.LayerChart(...)

🇺🇸 Pre-2016: Stable North American Trade under NAFTA (1994–2016)

NAFTA, signed in 1994, virtually eliminated tariffs on most agricultural products between the U.S., Mexico, and Canada.

Result: food trade grew steadily —

Canada remained the top exporter of grains, meats, and processed food,

Mexico specialized in fresh produce, beer.

Policy environment: predictable and pro-free trade; minimal political disruption.

🧩 In your chart: relatively smooth, parallel growth in U.S. imports from both countries through 2016.

🏛 2016–2020: Trump Administration — “America First” Trade Policy
🔹 2016 Campaign & Early 2017

Trump campaigned on NAFTA being a “bad deal” and promised to renegotiate or withdraw.

January 2017: U.S. withdrew from TPP (Trans-Pacific Partnership) — signaling a pivot from multilateral to bilateral trade.

April 2017: ordered a NAFTA renegotiation.

Created short-term uncertainty: some Canadian exports slowed, and importers diversified toward Mexico.

🔹 2017–2018: NAFTA Renegotiation Begins

The administration imposed tariffs on steel and aluminum (25% and 10%) on Canada and Mexico.

Canada retaliated with tariffs on U.S. dairy and food products.

Result: rising tension and temporary slowdown in some bilateral food flows.

Negotiations aimed to favor U.S. farmers, but the uncertainty caused many firms to shift sourcing to Mexico (cheaper, flexible supply).

🔹 2018–2019: USMCA Agreement & China Trade War

USMCA signed in late 2018, ratified 2019, took effect July 2020.

Expanded U.S. dairy access to Canada (a small gain for U.S. exporters).

Preserved duty-free trade for most agri-food.

Added stricter rules of origin and labor standards, pushing manufacturers (esp. processed foods) to relocate some production to Mexico.

Simultaneously: U.S.–China trade war began in 2018.

Tariffs on Chinese foods and ingredients (seafood, soy, garlic, etc.)

U.S. importers substituted Chinese goods with Mexican/Canadian suppliers — especially Mexico due to cost advantage.

🧩 In your chart: 2018–2019 often marks an inflection — Mexico’s export line climbing, Canada’s flattening.

🔹 2020: COVID-19 & USMCA Implementation

USMCA went into effect July 1 2020, solidifying North American integration.

Pandemic border controls temporarily disrupted Canadian trucking and logistics.

Mexico’s fresh produce sector rebounded faster, keeping exports high while Canadian shipments lagged due to cold-storage and labor issues.

🧩 In your chart: short dip or divergence around 2020–2021, then Mexico widening its lead.

📈 2021–2024: Biden Administration — Stability & Re-regionalization

Biden maintained USMCA and tariff structures; no return to TPP.

Focus shifted to supply chain resilience and “friend-shoring” — strengthening trade within the Americas.

Inflation and strong dollar increased nominal import values, even if real quantities were stable.

Canada’s exports stabilized; Mexico continued gradual rise in categories like produce, beverages, and processed foods.

In [45]:
top10_canada = (
    df_clean_b
    .filter(pl.col("Country") == "CANADA")
    .group_by("Commodity")
    .agg(pl.col("FoodValue_B").sum().alias("TotalImports_B"))
    .sort("TotalImports_B", descending=True)
    .head(10)
)

top10_mexico = (
    df_clean_b
    .filter(pl.col("Country") == "MEXICO")
    .group_by("Commodity")
    .agg(pl.col("FoodValue_B").sum().alias("TotalImports_B"))
    .sort("TotalImports_B", descending=True)
    .head(10)
)

chart_canada = (
    alt.Chart(top10_canada)
    .mark_bar(color="#C8102E")
    .encode(
        x=alt.X("TotalImports_B:Q", title="Total Food Imports (Billion $)"),
        y=alt.Y("Commodity:N", sort='-x', title="Commodity"),
        tooltip=["Commodity", "TotalImports_B"]
    )
    .properties(
        title="Top 10 Food Commodities Imported from Canada (1999–2024)",
        width=400,
        height=300
    )
)

chart_mexico = (
    alt.Chart(top10_mexico)
    .mark_bar(color="#006847")
    .encode(
        x=alt.X("TotalImports_B:Q", title="Total Food Imports (Billion $)"),
        y=alt.Y("Commodity:N", sort='-x', title="Commodity"),
        tooltip=["Commodity", "TotalImports_B"]
    )
    .properties(
        title="Top 10 Food Commodities Imported from Mexico (1999–2024)",
        width=400,
        height=300
    )
)

chart_canada | chart_mexico


alt.HConcatChart(...)

🇲🇽 Mexico (Right side)

Dominated by fresh vegetables, fruits, and beer/beverages → clear agricultural & manufacturing advantage.

Strong presence in prepared or preserved vegetables — fits post-2016 trend of processed exports rising.

Cereal & bakery foods show smaller values — Mexico’s exports are more fresh/seasonal.

🇨🇦 Canada (Left side)

Dominated by fresh/chilled meats, grains, and prepared foods.

Stronger presence in chocolate, refined oils, and malt beer → indicates more industrial/processed food trade.

Suggests a pattern: Mexico = fresh agricultural, Canada = processed / industrial food.

In [46]:
df_with_period = df_clean_b.with_columns(
    pl.when(pl.col("YearNum") <= 2016)
    .then(pl.lit("Before_2016"))
    .otherwise(pl.lit("After_2016"))
    .alias("Period")
)

df_agg = (
    df_with_period
    .filter(pl.col("Country").is_in(["MEXICO", "CANADA"]))
    .group_by(["Country", "Commodity", "Period"])
    .agg(pl.col("FoodValue_B").sum().alias("TotalValue_B"))
)

df_pivot = (
    df_agg
    .pivot(index=["Country", "Commodity"], columns="Period", values="TotalValue_B")
)

df_shift = df_pivot.with_columns(
    (pl.col("After_2016").fill_null(0) - pl.col("Before_2016").fill_null(0)).alias("Change")
)

selected_commodities = [
    "Fresh or chilled fruit",
    "Liquors and liqueurs",
    "Malt beer",
    "Refined vegetable oils",
    "Cereal and bakery foods",
    "Fresh or chilled red meats",
    "Wheat and products",
    "Bulk grains",
    "Bovine animals, live"
]

df_shift_selected = (
    df_shift
    .filter(pl.col("Commodity").is_in(selected_commodities))
    .with_columns(
        pl.when(pl.col("Change") > 0)
          .then(pl.lit("Increase"))
          .otherwise(pl.lit("Decrease"))
          .alias("Direction")
    )
)

/var/folders/j1/7ggvvb311jl0d3c1xxls5rrh0000gn/T/ipykernel_72297/176458755.py:16: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  df_agg


In [47]:
base = (
    alt.Chart(df_shift_selected)
    .transform_fold(
        ["Before_2016", "After_2016"], as_=["Period", "Value"]
    )
    .encode(
        alt.X("Value:Q").title("Import Value ($B)"),
        alt.Y("Commodity:N").title("Commodity")
    )
    .properties(width=350, height=300)
)

line = (
    base.mark_line(strokeWidth=3)
    .encode(
        detail=["Commodity:N", "Country:N"],
        stroke=alt.Stroke(  # Use stroke for lines
            "Direction:N",
            scale=alt.Scale(
                domain=["Increase", "Decrease"],
                range=["#08ec08df", "#e40d0d"]
            ),
            legend=alt.Legend(title="Change Direction")
        ),
        color=alt.value(None)  # Remove color encoding
    )
)

point = (
    base.mark_point(filled=True, size=90, stroke="black", strokeWidth=0.3)
    .encode(
        fill=alt.Fill(
            "Period:N",
            scale=alt.Scale(
                domain=["Before_2016", "After_2016"],
                range=["#9dbee8", "#dae39f"]
            ),
            legend=alt.Legend(title="Time Period")
        ),
        color=alt.value(None)
    )
)

chart_final = (
    (line + point)
    .facet(column=alt.Column("Country:N", title=None))
    .properties(
        title="Before vs After 2016: U.S. Food Imports by Commodity and Country"
    )
)
chart_final


alt.FacetChart(...)

🇲🇽 Mexico

Large positive shifts for:

Fresh fruits and vegetables, tree nuts, beer, bakery foods, etc.

These are labor-intensive or climate-dependent products that the U.S. increasingly sources from Mexico instead of Asia.

→ Reflects supply-chain “nearshoring” and year-round produce demand after trade tensions with China.

🇨🇦 Canada

Positive shifts for:

Red meats, dairy, cereal products → aligns with Canada’s strong agri-processing sector.

Declines for some fresh goods — climate limits Canada’s seasonal crops.

→ Suggests specialization in processed and animal-based exports rather than produce.

🧩 Takeaway:

After 2016, U.S. food imports realigned within North America —

Mexico gained ground in produce, beverages, and light manufacturing.

Canada consolidated its position in processed and high-value food commodities.

So this chart is visual evidence of supply-chain reorganization under post-2016 trade policy — fewer imports from China → increased intra-NAFTA trade.

After 2016, the U.S. government imposed 25% tariffs on most goods from Mexico and Canada,
except:

Canadian oil and energy exports → only 10% tariff.

Mexican energy exports → full 25%.

That means:

Goods with energy-related inputs (e.g., food processing, fertilizers, transportation-heavy commodities)
→ will show different responses between the two partners.

Price-sensitive agricultural imports → U.S. may import less after 2016.

Commodities exempt or strategically important (like oil-linked categories) → likely stable or even grew.

| Observation                                                                 | Likely Interpretation                                                                                    | Policy Connection                                       |
| --------------------------------------------------------------------------- | -------------------------------------------------------------------------------------------------------- | ------------------------------------------------------- |
| 🇲🇽 Fresh fruits, vegetables, beer, bakery goods → **increase after 2016** | Despite tariffs, U.S. demand remained high — food perishability and proximity make Mexico indispensable. | *Tariffs had limited effect; “nearshoring resilience.”* |
| 🇨🇦 Red meats, dairy, cereals → **increase**                               | Processed and energy-intensive goods stayed competitive — benefited from **lower (10%) energy tariffs**. | *Partial exemption = advantage to Canadian processors.* |
| 🇲🇽 Processed or energy-dependent goods → **decline**                      | Higher transportation and energy costs from 25% tariff plus energy-price rise.                           | *Direct negative impact of tariff policy.*              |


In [70]:
commodity_totals = (
    df_clean_b.group_by("Commodity")
      .agg(pl.col("FoodValue_B").sum().alias("TotalValue_B"))
)

df_dependency = (
    df_clean_b.join(commodity_totals, on="Commodity")
      .with_columns(
          (pl.col("FoodValue_B") / pl.col("TotalValue_B") * 100)
          .alias("Dependency_Ratio")
      )
)

risk_data = df_dependency.to_pandas()


top_commodities = (
    df_dependency.group_by("Commodity")
    .agg(pl.col("Dependency_Ratio").mean().alias("Avg_Dependency"))
    .sort("Avg_Dependency", descending=True)
    .head(15)
    ["Commodity"]
    .to_list()
)

risk_data_top = risk_data[risk_data["Commodity"].isin(top_commodities)]

risk_matrix = (
    alt.Chart(risk_data_top)
    .mark_rect()
    .encode(
        x=alt.X("Country:N", title="Source Country"),
        y=alt.Y("Commodity:N", title="Essential Commodity", sort="-x"),
        color=alt.Color("Dependency_Ratio:Q",
                        scale=alt.Scale(scheme="reds"),
                        legend=alt.Legend(title="Import Dependency (%)")),
    )
    .properties(width=450, height=300,
                title="Top 15 U.S. Food Commodities by Import Dependency")
)
risk_matrix


alt.Chart(...)

In [ ]:
chart_final.save('../static-viz/static-viz.html')